In [1]:
#Imports
import os
from random import uniform
import subprocess
from pathlib import Path
from nufeb_tools import utils,plot
import pandas as pd
from string import Template
import numpy as np
import pickle
from functools import partial
import time
from functools import partial
from skopt import gp_minimize, forest_minimize, dummy_minimize
import matplotlib.pyplot as plt

In [2]:
from tqdm.notebook import tqdm

class tqdm_skopt(object):
    def __init__(self, **kwargs):
        self._bar = tqdm(**kwargs)
        
    def __call__(self, res):
        self._bar.update()

In [3]:
def func(x):

    alpha = x[0]
    beta = x[1]
    delta = x[2]
    mu = x[3]
    rho = x[4]
    exp_low = [1.38,.041872]
    exp_high = [1.146667,1.141355]
    #TEMPLATES_DIR = (Path(__file__).parent) / 'templates'
    #Define inputs


    #Change input params
    HOME = Path.home()
    NUFEB_DIR = HOME / 'NUFEB/src/USER-NUFEB/'
    os.chdir('/tmp/NUFEB')
    filein = open( f'fix_bio_kinetics_monod.txt' )
            #read it

    #filein = open( TEMPLATES_DIR / 'fix_bio_kinetics_monod.txt' )
            #read it
    src = Template( filein.read() )
            #do the substitution
    result = src.safe_substitute({'alpha' : alpha, 'beta' : beta, 'delta' : delta,
                                        
                                        })
    #os.chdir(HOME)
    with open("/tmp/NUFEB/src/USER-NUFEB/fix_bio_kinetics_monod.cpp","w") as f:
       f.writelines(result)
    #Compile NUFEB
    #os.chdir(str(HOME / 'NUFEB'))
    compile_nufeb = subprocess.run('./comp.sh', stdout=subprocess.DEVNULL)
    #Clean old simulations
    os.system('nufeb-clean')
    #Run simulation
    #os.chdir('/lustre/or-scratch/cades-cnms/12x')
    text = f'nufeb-seed --cells 100,0 --d 1e-4,1e-4,1e-4 --grid 20 --t 8700 --mucya {mu} --sucR 0 --rhocya {rho}'
    os.system(text)
    text = f'nufeb-seed --cells 100,0 --d 1e-4,1e-4,1e-4 --grid 20 --t 8700 --mucya {mu} --sucR 1 --rhocya {rho}'
    os.system(text)
    #os.chdir(str(HOME / 'NUFEB'))
    run_nufeb = subprocess.run('./optim.sh', stdout=subprocess.DEVNULL)
    BASE_DIR = Path(f'runs/')
    folders = [path for path in BASE_DIR.iterdir() if path.is_dir()]

    #os.system(r'for dir in runs/*/;do;cd "$dir";mpirun -np 32 $LAMMPS -in *.lammps > nufeb.log;cd "$base";done')

    #Extract output

    data = [utils.get_data(directory=str(x)) for x in folders]
    Volume = 1e-4*1e-4*1e-4 #m^3
    CellNum2OD = Volume*1e6/0.3e-8
    SucroseMW = 342.3
    dfs = []
    for x in data:
        temp = pd.concat([x.ntypes.cyano/CellNum2OD,x.ntypes.step/60/60*x.timestep,x.avg_con.Sucrose.reset_index(drop=True)/SucroseMW*1e3],axis=1)
        temp.columns=['OD750','Hours','Sucrose']
        temp['SucroseExport'] = x.sucRatio/100
        dfs.append(temp)
    df = pd.concat(dfs)
    low_suc = df.loc[(df.Hours > 23.8) & (df.Hours < 24) & (df.SucroseExport==0)].mean()[['OD750','Sucrose']].to_numpy()
    high_suc = df.loc[(df.Hours > 23.8) & (df.Hours < 24) & (df.SucroseExport==1)].mean()[['OD750','Sucrose']].to_numpy()

    #Compare output with experimental data
    residual = np.sqrt((low_suc - exp_low)**2).sum() + np.sqrt((high_suc - exp_high)**2).sum()
    print(residual)
    return residual
    #Optimize


In [4]:
bounds = [(0.1, .5),(1,5),(0.01,.1),(0.000001,0.00001),(320,390)]
n_calls = 50

def run(minimizer, n_iter=1):
    return [minimizer(func, bounds, n_calls=n_calls, random_state=n,callback=[tqdm_skopt(total=n_calls)])
            for n in range(n_iter)]
#et_res = forest_minimize(func, bounds, n_calls=n_calls,callback=[tqdm_skopt(total=n_calls, desc="Random forest")],base_estimator="ET")

In [5]:
# Random search
dummy_res = run(dummy_minimize)

# Random forest
rf_res = run(partial(forest_minimize, base_estimator="RF"))

# Extra trees
et_res = run(partial(forest_minimize, base_estimator="ET"))

2.430875060139564
2.4843096888066833
2.0644056534375173
2.0208183799142247
2.083818321998878
2.4282349430064825
2.037000594980989
2.302842392364586
1.85075512165005
2.8730411592940435
2.512103381314848
2.575046819329536
2.7582859839604725
2.6112999039702296
1.7693350106798462
4.344472252393096
2.2856437068557813
2.039224440169092
2.524064169951017
2.7652668220935706
2.0858098031434684
2.485685560143447
2.3192949838443164
2.291116331551211
1.998635822727952
3.159120933630305
1.6841682461983
2.2429391583345772
2.0563514881353298
1.8249523333738729
2.0873817635798604
3.4112527959979957
2.467031589465636
2.2980990470733413
2.249253198045836
2.256848335474765
2.94687036684493
2.1638154281318935
2.545034863039624
2.2146214273830624
1.621123693448172
3.0136433230721122
2.141337949519537
2.433545854528947
1.6453894293257216
2.8286279770514953
2.240493351417082
2.6757117914195776
2.8276848759197906
2.314245829461798


2.5298074915812836
2.4298726612374284
2.018446946254091
2.1476570559871107
2.121767172077257
2.4445368114397548
2.0535280025465714
2.2851403209429706
1.7839474338164887
2.8349563037828425
2.2861215663899643


KeyboardInterrupt: 

In [ ]:
plot = plot_convergence(("dummy_minimize", dummy_res),
                        ("forest_minimize('rf')", rf_res),
                        ("forest_minimize('et)", et_res),
                        true_minimum=1.4)

plot.legend(loc="best", prop={'size': 6}, numpoints=1)

In [ ]:
plot_objective(dummy_res)

In [ ]:
plot_objective(rf_res)

In [ ]:
plot_objective(et_res)